In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cupy as cp
import pickle
import lstm
from scipy import stats
import copy

In [2]:
mice = ["mpfc14", "mpfc19", "mpfc20", "mpfc23", "mpfc24", "mpfc30"]
days = ["yd1", "yd2", "yd6"]

In [3]:
with open("h_spike_Wlim001.pkl", "rb") as f:
    h = pickle.load(f)

In [4]:
beh = []
n_beh = []
for mi, mouse in enumerate(mice):
    behs = []
    n_behs = []
    for di, day in enumerate(days):
        df = pd.read_csv("behavior/" + mouse + "/" +mouse + "_" + day + ".csv", header=None).values
        
        n = np.sum(~np.isnan(df).any(axis=1))
        df = df[:n]
        t = np.zeros(n)
        for i in range(1, n-1):
            if df[i, 0] == 1:
                t[i] = 1
            else:
                if df[i-1, 0] == 1 and df[i+1, 0] == 1:
                    t[i] = 1
        df[:, 0] = t.copy()
        
        # remove "just-in-case" lick
        count = 0
        for i in range(n):
            if df[i, 2] == 1:
                count = 0
            elif df[i, 1] == 1:
                if count < 50:
                    df[i, 1] = 0
                count = 0
            else:
                count += 1
                
        t = df.shape[0]
        t_train = int(t * 0.5)
        start = (t - t_train) // 2
        end = start + t_train
        
        behs.append(df)
        n_behs.append((start, end))
    beh.append(behs)
    n_beh.append(n_behs)

In [5]:
n_beh

[[(4513, 13540), (3382, 10146), (2018, 6053)],
 [(5767, 17301), (5036, 15109), (3573, 10720)],
 [(4949, 14847), (4541, 13623), (2726, 8178)],
 [(4630, 13890), (5074, 15222), (3204, 9612)],
 [(5321, 15962), (4562, 13686), (2264, 6793)],
 [(4513, 13540), (4461, 13382), (2301, 6902)]]

In [6]:
batch_size = 10
lr = 10
max_epoch = 100
max_iters = 10
max_effect = 100
min_effect = 50
max_grad = 0.01
padding = 49
params = []
params_all = []

for mi, mouse in enumerate(mice):
    params_s = []
    params_all_s = []
    for di, day in enumerate(days):
        start, end = n_beh[mi][di]
        n = beh[mi][di].shape[0]
        x_train = h[mi][di][padding+start:padding+end]
        input_size = x_train.shape[1]
        y_train = beh[mi][di][start:end]
        y_test = np.vstack((beh[mi][di][:start], beh[mi][di][end:]))
        output_size = y_train.shape[1]
        
        rate = [
            (len(y_train[:, 0]) - y_train[:, 0].sum()) / y_train[:, 0].sum(),
            (len(y_train[:, 1]) - y_train[:, 1].sum()) / y_train[:, 1].sum(),
            (y_train[:, 1].sum() - y_train[:, 2].sum()) / y_train[:, 2].sum(),
        ]

        err_min = np.inf
        param_min = None
        params_all_ss = []
        for i in range(10):
            print(mouse, day, "iter:", i)
            model = lstm.Rnnlm(input_size, output_size=output_size, rate=rate)
            optimizer = lstm.SGD(lr)
            trainer = lstm.RnnlmTrainer(model, optimizer, seed=i*100+mi*10+di)
            trainer.fit(x_train, y_train, max_epoch, max_iters, batch_size, max_effect, min_effect, max_grad, save_params=True)
    
            e_train = np.zeros((max_epoch, 5))
            e_test = np.zeros((max_epoch, 5))
            xy_rate_train = np.square(y_train[:, 3:] - y_train.mean(axis=0)[3:]).mean(axis=0) / 0.25
            xy_rate_test = np.square(y_test[:, 3:] - y_test.mean(axis=0)[3:]).mean(axis=0) / 0.25

            for j in range(max_epoch):
                decoder = lstm.Decoder(trainer.params[j])
                pred = decoder.predict(h[mi][di][:padding+n])
                err = np.square(pred[49:] - beh[mi][di])
                num = np.ones_like(err)
                
                num[beh[mi][di][:, 0] == 1, 0] *= rate[0]
                err[beh[mi][di][:, 0] == 1, 0] *= rate[0]
                num[beh[mi][di][:, 1] == 1, 1] *= rate[1]
                err[beh[mi][di][:, 1] == 1, 1] *= rate[1]
                num[beh[mi][di][:, 2] == 1, 2] *= rate[2]
                err[beh[mi][di][:, 2] == 1, 2] *= rate[2]
                num[beh[mi][di][:, 1] == 1, 2] = 0
                err[beh[mi][di][:, 1] == 1, 2] = 0
                
                err_train = err[start:end]
                err_test = np.vstack((err[:start], err[end:]))
                num_train = num[start:end]
                num_test = np.vstack((num[:start], num[end:]))
    
                e_train[j, :] = err_train.sum(axis=0) / num_train.sum(axis=0)
                e_test[j, :] = err_test.sum(axis=0) / num_test.sum(axis=0)
                e_train[j, 3:] /= xy_rate_train
                e_test[j, 3:] /= xy_rate_test
                print(j, ":", e_train[j, :].sum(), e_test[j, :].sum())
    
            index = e_test.sum(axis=1).argmin()
            e_index = e_train[index].sum()
            print("iter:", i, "index:", index)
            if err_min > e_index:
                err_min = e_index
                param_min = [param.copy() for param in trainer.params[index]]
            params_all_ss.append([param.copy() for param in trainer.params[index]])
    
        params_s.append(param_min)
        params_all_s.append(params_all_ss)
    params.append(params_s)
    params_all.append(params_all_s)

mpfc14 yd1 iter: 0
epoch 0 / 100 | time 0[s] | whe: 0.12, lic: 0.12, rew: 0.13, x: 0.03, y: 0.01 
epoch 1 / 100 | time 0[s] | whe: 0.12, lic: 0.12, rew: 0.11, x: 0.03, y: 0.01 
epoch 2 / 100 | time 1[s] | whe: 0.12, lic: 0.12, rew: 0.04, x: 0.03, y: 0.01 
epoch 3 / 100 | time 1[s] | whe: 0.12, lic: 0.12, rew: 0.04, x: 0.03, y: 0.01 
epoch 4 / 100 | time 2[s] | whe: 0.11, lic: 0.12, rew: 0.05, x: 0.03, y: 0.01 
epoch 5 / 100 | time 2[s] | whe: 0.11, lic: 0.12, rew: 0.11, x: 0.03, y: 0.01 
epoch 6 / 100 | time 3[s] | whe: 0.12, lic: 0.11, rew: 0.07, x: 0.03, y: 0.01 
epoch 7 / 100 | time 3[s] | whe: 0.10, lic: 0.11, rew: 0.13, x: 0.03, y: 0.01 
epoch 8 / 100 | time 3[s] | whe: 0.12, lic: 0.11, rew: 0.10, x: 0.03, y: 0.01 
epoch 9 / 100 | time 4[s] | whe: 0.10, lic: 0.11, rew: 0.14, x: 0.03, y: 0.01 
epoch 10 / 100 | time 4[s] | whe: 0.11, lic: 0.10, rew: 0.08, x: 0.03, y: 0.01 
epoch 11 / 100 | time 5[s] | whe: 0.10, lic: 0.10, rew: 0.08, x: 0.03, y: 0.01 
epoch 12 / 100 | time 5[s] | wh

3 : 1.0316103252415636 1.110437698038087
4 : 1.0148110838880537 1.0763746665176381
5 : 1.0199074877128806 1.0661620688638211
6 : 1.0006538905882918 1.0827052997414615
7 : 1.0380738244269223 1.1603451929944308
8 : 0.9983731445597395 1.0597241359775045
9 : 1.0352579382612017 1.1095854308777664
10 : 1.0493419194381337 1.1262820902416335
11 : 1.0576912418342803 1.1456520649108144
12 : 1.0762118663150468 1.1785155778014949
13 : 1.0691998733180625 1.1771192911038502
14 : 1.0419155703667708 1.1491693070688787
15 : 1.085506494117209 1.234254458985946
16 : 1.041736646608159 1.1799210088556213
17 : 1.0764086231534555 1.2238697383578039
18 : 1.085574047499445 1.228103959841174
19 : 1.1131601108274711 1.2173106282973067
20 : 1.0419290884114325 1.1752028345742553
21 : 0.9954719816821206 1.084528138595677
22 : 0.9760240979326246 1.1232980348844346
23 : 1.0021558936414237 1.1327429214312048
24 : 0.9968352567995846 1.1837801022068197
25 : 1.046324307184674 1.2386102736068159
26 : 0.9743233001860272 1.

epoch 51 / 100 | time 24[s] | whe: 0.07, lic: 0.04, rew: 0.01, x: 0.03, y: 0.01 
epoch 52 / 100 | time 24[s] | whe: 0.10, lic: 0.07, rew: 0.01, x: 0.03, y: 0.01 
epoch 53 / 100 | time 25[s] | whe: 0.10, lic: 0.06, rew: 0.05, x: 0.03, y: 0.01 
epoch 54 / 100 | time 25[s] | whe: 0.12, lic: 0.06, rew: 0.01, x: 0.03, y: 0.01 
epoch 55 / 100 | time 26[s] | whe: 0.09, lic: 0.06, rew: 0.00, x: 0.03, y: 0.01 
epoch 56 / 100 | time 26[s] | whe: 0.10, lic: 0.05, rew: 0.04, x: 0.03, y: 0.01 
epoch 57 / 100 | time 27[s] | whe: 0.11, lic: 0.05, rew: 0.09, x: 0.03, y: 0.01 
epoch 58 / 100 | time 27[s] | whe: 0.11, lic: 0.05, rew: 0.01, x: 0.03, y: 0.01 
epoch 59 / 100 | time 28[s] | whe: 0.09, lic: 0.05, rew: 0.04, x: 0.03, y: 0.01 
epoch 60 / 100 | time 28[s] | whe: 0.10, lic: 0.05, rew: 0.02, x: 0.03, y: 0.01 
epoch 61 / 100 | time 29[s] | whe: 0.09, lic: 0.05, rew: 0.03, x: 0.03, y: 0.01 
epoch 62 / 100 | time 29[s] | whe: 0.11, lic: 0.04, rew: 0.03, x: 0.03, y: 0.01 
epoch 63 / 100 | time 30[s] 

epoch 0 / 100 | time 0[s] | whe: 0.13, lic: 0.12, rew: 0.11, x: 0.03, y: 0.02 
epoch 1 / 100 | time 0[s] | whe: 0.12, lic: 0.12, rew: 0.07, x: 0.03, y: 0.01 
epoch 2 / 100 | time 1[s] | whe: 0.11, lic: 0.12, rew: 0.07, x: 0.03, y: 0.01 
epoch 3 / 100 | time 1[s] | whe: 0.12, lic: 0.12, rew: 0.09, x: 0.03, y: 0.01 
epoch 4 / 100 | time 2[s] | whe: 0.11, lic: 0.12, rew: 0.02, x: 0.03, y: 0.01 
epoch 5 / 100 | time 2[s] | whe: 0.13, lic: 0.12, rew: 0.09, x: 0.03, y: 0.01 
epoch 6 / 100 | time 3[s] | whe: 0.12, lic: 0.12, rew: 0.12, x: 0.03, y: 0.01 
epoch 7 / 100 | time 3[s] | whe: 0.11, lic: 0.11, rew: 0.04, x: 0.03, y: 0.01 
epoch 8 / 100 | time 4[s] | whe: 0.10, lic: 0.12, rew: 0.09, x: 0.03, y: 0.01 
epoch 9 / 100 | time 4[s] | whe: 0.11, lic: 0.10, rew: 0.07, x: 0.03, y: 0.01 
epoch 10 / 100 | time 4[s] | whe: 0.11, lic: 0.11, rew: 0.08, x: 0.03, y: 0.01 
epoch 11 / 100 | time 5[s] | whe: 0.11, lic: 0.11, rew: 0.05, x: 0.03, y: 0.01 
epoch 12 / 100 | time 5[s] | whe: 0.11, lic: 0.11,

4 : 1.0439853695984804 1.1124471911568625
5 : 1.0204345986779628 1.0715900925361757
6 : 1.0315967761271327 1.1084498369063873
7 : 1.0113141487604647 1.0850894740570318
8 : 1.0104005492067252 1.090780861301623
9 : 1.0083118402574742 1.0888917658694695
10 : 1.0245523622735961 1.119759415644946
11 : 0.9884680587229465 1.0791151112218615
12 : 1.017708875333367 1.1134573671514787
13 : 0.9884198325517198 1.0857426963133952
14 : 0.966375408118922 1.038797245579522
15 : 1.0111674961055406 1.1165986924154547
16 : 0.9853278459843742 1.0905979528825953
17 : 0.967372895985193 1.0593981090653817
18 : 1.0341669530454976 1.1547270381207495
19 : 1.0555690812446878 1.1670058676439026
20 : 1.190471940682786 1.2886641356185513
21 : 1.0295346053588441 1.0571916159717734
22 : 1.021145415826688 1.0663348134311443
23 : 0.9911819893921601 1.0633476578472192
24 : 1.1045759449871064 1.2086060782688905
25 : 1.0893045819267544 1.1918141121508712
26 : 1.0047307734447135 1.0701098069008044
27 : 1.0706663988524703 1

...

epoch 17 / 100 | time 8[s] | whe: 0.08, lic: 0.10, rew: 0.07, x: 0.02, y: 0.02 
epoch 18 / 100 | time 8[s] | whe: 0.08, lic: 0.09, rew: 0.09, x: 0.02, y: 0.01 
epoch 19 / 100 | time 9[s] | whe: 0.07, lic: 0.08, rew: 0.09, x: 0.02, y: 0.02 
epoch 20 / 100 | time 9[s] | whe: 0.08, lic: 0.08, rew: 0.09, x: 0.02, y: 0.02 
epoch 21 / 100 | time 10[s] | whe: 0.07, lic: 0.09, rew: 0.08, x: 0.02, y: 0.02 
epoch 22 / 100 | time 10[s] | whe: 0.07, lic: 0.07, rew: 0.09, x: 0.02, y: 0.01 
epoch 23 / 100 | time 11[s] | whe: 0.08, lic: 0.09, rew: 0.07, x: 0.02, y: 0.02 
epoch 24 / 100 | time 11[s] | whe: 0.07, lic: 0.08, rew: 0.09, x: 0.02, y: 0.02 
epoch 25 / 100 | time 12[s] | whe: 0.07, lic: 0.08, rew: 0.08, x: 0.02, y: 0.01 
epoch 26 / 100 | time 12[s] | whe: 0.07, lic: 0.08, rew: 0.09, x: 0.02, y: 0.01 
epoch 27 / 100 | time 13[s] | whe: 0.07, lic: 0.07, rew: 0.07, x: 0.02, y: 0.02 
epoch 28 / 100 | time 13[s] | whe: 0.06, lic: 0.08, rew: 0.08, x: 0.02, y: 0.02 
epoch 29 / 100 | time 14[s] | wh

35 : 0.6801522344247705 0.7075139834820454
36 : 0.6868380922673095 0.7138436176943085
37 : 0.6875972638622456 0.7030998952205205
38 : 0.728161778071235 0.7333167452463758
39 : 0.7053146154866252 0.7436450888243502
40 : 0.691565013949195 0.7267814713430734
41 : 0.7103988548310046 0.7394778384320465
42 : 0.7414637791689093 0.7542962914568604
43 : 0.6983323294409896 0.7355675950336096
44 : 0.7015134582402481 0.7358535033072291
45 : 0.6980985415313681 0.7435045390521281
46 : 0.7031718520607497 0.7365512479874877
47 : 0.7637906853263735 0.7698757703265582
48 : 0.6551431811354353 0.7041514961758946
49 : 0.6574712974938821 0.7022106808156934
50 : 0.6734924934153277 0.7056308641499105
51 : 0.6833468894962551 0.7092163150163143
52 : 0.6631241015379019 0.693799577848833
53 : 0.6499120243911115 0.6876510602025463
54 : 0.6499562569957126 0.6874164443180818
55 : 0.6482250888485085 0.6886836134612025
56 : 0.6967093613889584 0.7202725036142819
57 : 0.6469531212471771 0.6869851205947036
58 : 0.6449016

epoch 67 / 100 | time 35[s] | whe: 0.07, lic: 0.05, rew: 0.06, x: 0.02, y: 0.02 
epoch 68 / 100 | time 36[s] | whe: 0.08, lic: 0.07, rew: 0.08, x: 0.02, y: 0.02 
epoch 69 / 100 | time 36[s] | whe: 0.08, lic: 0.08, rew: 0.05, x: 0.02, y: 0.02 
epoch 70 / 100 | time 37[s] | whe: 0.08, lic: 0.08, rew: 0.07, x: 0.02, y: 0.02 
epoch 71 / 100 | time 38[s] | whe: 0.07, lic: 0.07, rew: 0.04, x: 0.02, y: 0.02 
epoch 72 / 100 | time 38[s] | whe: 0.07, lic: 0.05, rew: 0.06, x: 0.02, y: 0.02 
epoch 73 / 100 | time 39[s] | whe: 0.07, lic: 0.07, rew: 0.06, x: 0.02, y: 0.02 
epoch 74 / 100 | time 39[s] | whe: 0.07, lic: 0.07, rew: 0.07, x: 0.01, y: 0.02 
epoch 75 / 100 | time 40[s] | whe: 0.06, lic: 0.06, rew: 0.08, x: 0.02, y: 0.02 
epoch 76 / 100 | time 40[s] | whe: 0.07, lic: 0.07, rew: 0.07, x: 0.02, y: 0.02 
epoch 77 / 100 | time 41[s] | whe: 0.08, lic: 0.07, rew: 0.07, x: 0.02, y: 0.02 
epoch 78 / 100 | time 42[s] | whe: 0.07, lic: 0.06, rew: 0.07, x: 0.01, y: 0.02 
epoch 79 / 100 | time 42[s] 

epoch 16 / 100 | time 7[s] | whe: 0.08, lic: 0.07, rew: 0.08, x: 0.02, y: 0.02 
epoch 17 / 100 | time 7[s] | whe: 0.08, lic: 0.08, rew: 0.08, x: 0.02, y: 0.02 
epoch 18 / 100 | time 8[s] | whe: 0.08, lic: 0.06, rew: 0.08, x: 0.02, y: 0.02 
epoch 19 / 100 | time 8[s] | whe: 0.08, lic: 0.08, rew: 0.09, x: 0.02, y: 0.01 
epoch 20 / 100 | time 9[s] | whe: 0.07, lic: 0.08, rew: 0.09, x: 0.02, y: 0.02 
epoch 21 / 100 | time 9[s] | whe: 0.07, lic: 0.07, rew: 0.07, x: 0.02, y: 0.01 
epoch 22 / 100 | time 10[s] | whe: 0.07, lic: 0.08, rew: 0.09, x: 0.02, y: 0.01 
epoch 23 / 100 | time 10[s] | whe: 0.06, lic: 0.09, rew: 0.07, x: 0.02, y: 0.01 
epoch 24 / 100 | time 11[s] | whe: 0.05, lic: 0.07, rew: 0.07, x: 0.02, y: 0.01 
epoch 25 / 100 | time 11[s] | whe: 0.04, lic: 0.08, rew: 0.06, x: 0.02, y: 0.01 
epoch 26 / 100 | time 12[s] | whe: 0.06, lic: 0.07, rew: 0.07, x: 0.02, y: 0.01 
epoch 27 / 100 | time 12[s] | whe: 0.06, lic: 0.09, rew: 0.08, x: 0.02, y: 0.01 
epoch 28 / 100 | time 13[s] | whe:

33 : 0.6946531512259367 0.6350159573821722
34 : 0.6795569619200591 0.6617988767696846
35 : 0.689977004930929 0.6288973504059328
36 : 0.6771880965045326 0.6450590443387264
37 : 0.6656092258672107 0.6535133945222028
38 : 0.6886456215944755 0.6436225032365234
39 : 0.6842557926244304 0.659107150858664
40 : 0.7220993129838155 0.6508239158217053
41 : 0.6975484077820633 0.6396719263804801
42 : 0.6982603491388032 0.6362623956731227
43 : 0.6848518491232373 0.6315317943946949
44 : 0.6615189292321564 0.6221156617732867
45 : 0.6758360241254103 0.6184335494522064
46 : 0.6659294235372212 0.6409561391874888
47 : 0.6686492886698996 0.6241823014315386
48 : 0.6544896500988608 0.6376072695882282
49 : 0.6566263150086022 0.6430466064297453
50 : 0.6545088219362603 0.6513653258918964
51 : 0.6536580987608039 0.6439222088658911
52 : 0.7330453654578577 0.7952669038269494
53 : 0.70083228604289 0.6980769128562604
54 : 0.6833650664228546 0.6872069274378056
55 : 0.6819587675141741 0.6835182956233861
56 : 0.72893354

In [7]:
with open("params.pkl", "wb") as f:
    pickle.dump(params, f)
with open("params_all.pkl", "wb") as f:
    pickle.dump(params_all, f)
with open("n_beh.pkl", "wb") as f:
    pickle.dump(n_beh, f)